在前面的学习中，我们从同一份样本数据中得到了三个模型，而且三个模型的拟合效果都很好，不过很显然，二次模型和三次模型都过于复杂了，属于过拟合。为了判断哪个模型的性能最优，泛化能力最强，我们将数据集划分为训练集和测试集，然后选择在测试集上表现最好的模型。使用这种方法可以有效的评估模型结果，避免过拟合。

这一节我们将学习一种避免过拟合的新方法：**正则化**（regularization）方法。

在线性回归模型 $ \hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n = w^Tx $ 中，每一个变量 $x_i$ 就是模型的一个特征，参数 $w_i$ 代表相应的特征在模型中所占的权重。在前面的例子里，本来只有一个特征，我们人为造出了两个特征来（$x^2$ 和 $x^3$），得到了三个模型，这两个特征对模型来说其实根本没用，过拟合就是因为模型过于复杂，把这些没用的特征也学习到了。所以如果能降低这些没用特征的权重，譬如直接降到 0，那就不会出现过拟合了，为了达到这一点，我们可以稍微改进下我们的损失函数：

$$
loss = \sum_{i=1}^m (y_i - w^Tx_i)^2 + \lambda \| w \|^2
$$

这个函数相比于我们之前的平方损失函数多了一个 $\lambda \| w \|^2$，这被称之为 **正则化项** 或 **惩罚项**，其中正则化参数 $\lambda > 0$，它是一个超参数，$\| w \|^2$ 表示 $L_2$ 范数，如下：

$$
\| w \|^2 = \sum_{i=1}^m w_i^2 = w_1^2 + w_2^2 + \dots + w_m^2
$$

很显然，为了使上面的损失函数最小，模型中不相干的参数越小越好，从而降低过拟合的风险，使用这个损失函数的回归方法称为 **岭回归**（ridge regression）。

上面的 $L_2$ 范数可以改成 $L_p$ 范数，当 $p = 1$ 时，有损失函数：

$$
loss = \sum_{i=1}^m (y_i - w^Tx_i)^2 + \lambda \| w \|
$$

其中 $L_1$ 范数如下：

$$
\| w \| = \sum_{i=1}^m |w_i| = |w_1| + |w_2| + \dots + |w_m|
$$

这种回归方法称为 **LASSO 回归**（Least Absolute Shrinkage and Selection Operator）。

可以看到这里的损失函数仍然是最小二乘，但是在后面增加了 $L_1$ 或 $L_2$ 范数约束，所以岭回归又被称为 **基于 $L_2$ 约束的最小二乘法**，LASSO 回归又被称为 **基于 $L_1$ 约束的最小二乘法**。

### $L_2$ 约束的最小二乘

在普通最小二乘回归中，参数空间没有限制，参数可以取任何值，但是在岭回归和 Lasso 回归中，参数被限制在一定的范围内，这被称为 **部分空间约束的最小二乘学习法**。为了保证参数不会偏移到值域范围之外，可以附加一个 $P\theta = \theta$ 的约束条件。

$$
\mathop{\min}_{\theta}J(\theta), P\theta = \theta
$$

$P$ 是满足 $P^2 = P$ 和 $P^T = P$ 的 $b \times b$ 维矩阵，表示的是矩阵 $P$ 的值域的正交投影矩阵。$P$ 通常是手工设置的，通过主成分分析法，也可以基于数据进行设置。

不过正交投影矩阵 $P$ 的设置有很大的自由度，所以改用操作相对容易的 $L_2$ 约束条件，譬如：

$$
\mathop{\min}_{\theta}J(\theta), \|\theta\|^2 \leqslant R
$$

这时的参数空间如下图所示：

![](../images/l2-parameter-space.png)

从图中可以看出来，这种约束方法是以参数空间的原点为圆心，在一定半径范围的圆内（一般为**超球**）进行求解的。$R$ 表示的是圆的半径。

利用**拉格朗日对偶问题**，该问题可以转换为求解下式的最优解：

$$
\mathop{\max}_{\lambda} \mathop{\min}_{\theta} [ J(\theta) + \frac{\lambda}{2}(\|\theta\|^2 - R) ], \lambda \geqslant 0
$$

拉格朗日对偶问题的拉格朗日乘子 $\lambda$ 的解由圆的半径 $R$ 决定，如果不根据 $R$ 来决定 $\lambda$，而是直接指定的话，$L_2$ 约束的最小二乘解 $\hat{\theta}$ 可以通过下式求得：

$$
\hat{\theta} = \mathop{\arg\min}_{\theta} [ J(\theta) + \frac{\lambda}{2}\|\theta\|^2 ]
$$

这就是岭回归。

### 一般 $L_2$ 约束的最小二乘

上面的 $L_2$ 约束可以写成更一般的情况：

$$
\mathop{\min}_{\theta}J(\theta), \theta^TG\theta \leqslant R
$$

这里的 $G$ 是一个 $b \times b$ 的正则化矩阵。当 $G$ 是对称正定矩阵的时候，$\theta^TG\theta \leqslant R$ 可以把参数限制在一个椭圆区域内。

### 岭回归的求解

我们将岭回归的损失函数表示成矩阵形式：

$$
loss = (y - w^TX)^2 + \lambda w^2
$$

对 $w$ 求偏导：

$$
\frac {\partial}{\partial w}loss = 2X^T(Xw-y)+2 \lambda w
$$

令其等于 0，求得损失函数最小时 $w$ 的值：

$$
\hat{w} = (X^TX + \lambda I)^{-1}X^Ty
$$

### 什么时候使用岭回归？

我们知道，使用最小二乘进行线性回归时，有正规方程和梯度下降两种解法，使用正规方程可以求得回归系数：

$$
\hat{w} = (X^TX)^{-1}X^Ty
$$

在前面我们提到过，正规方程有很多不足之处，其中很重要的一点是 $X^TX$ 必须是可逆的，关于 $X^TX$ 可逆有很多其他的说法：

* $X^TX$ 必须是可逆的
* $X^TX$ 的行列式不能等于 0
* $X$ 必须是满秩的
* $X$ 是正定矩阵

为了保证 $X^TX$ 可逆，可以在 $X^TX$ 的对角线上加上一个很小的常量，这样可以提高其正则性，进而可以更稳定的进行逆矩阵的求解，这就是岭回归。

另一方面，$X^TX$ 不能为 0，当变量之间具有很强的相关性时，$X^TX$ 会变得很小，甚至趋于 0，譬如一个自变量是身高 $x_1$，另一个是体重 $x_2$，根据这两个变量我们得到一个回归模型，$y = ax_1 + bx_2 + c$，由于 $x_1$ 和 $x_2$ 高度相关，所以 a 和 b 之间存在互相抵消的效应：你可以把 a 弄成一个很大的正数，同时把 b 弄成一个绝对值很大的负数，最终 $\hat{y}$ 可能不会改变多少。这会导致用不同人群拟合出来的 a 和 b 差别可能会很大，模型的可解释性就大大降低了。这种情况被称为 **多重共线性**。

岭回归是一种专用于共线性数据分析的有偏估计回归方法，实质上是一种改良的最小二乘估计法，通过放弃最小二乘法的无偏性，以损失部分信息、降低精度为代价获得回归系数更为符合实际、更可靠的回归方法，对病态数据的拟合要强于最小二乘。

### 岭回归的几何解释

在一元线性回归的求解过程中，我们知道，损失函数实际上就是三维空间的一个曲面，损失函数的最小值就是该曲面的最低点。

![](../images/ridge-regression-1.webp)

岭回归意味着参数的取值被限制在一个圆内，也就是下面三维空间中的圆柱体，这个圆柱体和曲面的交点就是岭回归的解。

![](../images/ridge-regression-2.webp)

从 β1β2 平面理解，即为抛物面等高线在水平面的投影和圆的交点，如下图所示：

![](../images/ridge-regression-3.webp)

### 参考

* https://www.jianshu.com/p/1677d27e08a7